# Valuation Framework


## Fundamental Theorem of Asset Pricing

Then central notion underlying the theorem is the concept of a martingale measure; i.e., a probability measure that removes the drift from a discounted risk factor (stochastic process). Under a martingale measure, all risk factors drift with the risk-free short rate - and not with any other market rate involving some kind of risk premium over the risk-free short rate.

### A Simple Exmple

Condsider a simple economy at the dates today and tomorrow with a risky asset, a "stock" and a riskless asset, a "bond". The bond costs 10 USD today and pays off 10 USD tomorrow (zero interest rates). The stock costs 10 USD today and, with a probability of 60% and 40%, respectively, pays off 20 USD or 0 USD tomorrow. The riskless return of the bond is 0. The expected return of the stock is $\frac{0.6\times 20+0.4\times 0}{10}-1=0.2$, or 20%.

Consider now a call option with strike price of 15 USD. What is the fair value of such a contingent claim that pays 5 USD with 60% probability and 0 USD otherwise? Discounting the resulting value back yields a value of $0.6\times 5 = 3$ USD.

There is another approach that has been successfully applied to option pricing problems: replication of the option's payoff through a portfolio of traded securities. Buying 0.25 of the stock perfectly replicates the option's payoff (in the 60% case one has $0.25\times 20=5$ USD). A quater of the stock only costs 2.5 USD amd not 3 USD. Taking expectations under the real-world probability measure overvalues the option.

Why is this the case? The real-world measure implies a risk premium of 20 % for the stock since the risk involved in the stock is real in the sense that it cannot be diversified or hedged away.

Can one save the approach of taking expectations to value the call option? One only has to change the probability in such a way that the risky asset, the stock, drifts with the riskless short rate of zero. A martingale measure giving equal mass of 50% to both scenarios accomplishes this. Now, taking expectations of ths option's payoff under the new martingale measure yields the correct (arbitrage free) fair value : $0.5 \times 5 + 0.5\times 5 = 2.5$ USD.

### The General Results

Consider a general market model in discrete time:

A general maket model $\mathcal{M}$ in discrete time is a collection of:

- A finite state space $\Omega$

- A filtration $\mathbb{F}$

- A strictly positive probability measure $P$ defined on $\varsigma(\Omega)$

- A terminal date $T\in\mathbb{N}$, $T<\infty$

- A set $\mathbb{S}\equiv \{\left(S_t^k\right)_{t\in\{0,...,T\}}: k\in\{0,...,K\}\}$ of $K+1$ strictly positive security price processes

Together one has
$$\mathcal{M}=\{(\Omega, \varsigma(\Omega), \mathbb{F}, P), T, \mathbb{S}\}$$

Based on a general market model, one can formulate the Fundermental Theorem of Asset Pricing as follows:

Consider the general market model $\mathcal{M}$. Accoding to the Fundermental Theorem of Asset Pricing, the following three statements are equivalent:

- There are no arbitrage opportunities in the market model $\mathcal{M}$

- The set $\mathbb{Q}$ of $P$-equivalent martingale measures is nonempty.

- These set $\mathbb{P}$ of consistent linear price system is nonempty.

When it comes to valuation and pricing of contingent claims (i.e., option, derivatives, futures, forwards, swaps, etc), the importance of the theorem is illustrated by the following corollary:


If the market model $\mathcal{M}$ is arbitrage-free, then there exists a unique price $V_0$ associated with any attainable(i.e., replicable) contingent claim (option, derivative, etc) $V_T$. It satisfies $\forall Q\in \mathcal{Q}: V_0=\mathbf{E}_0^Q(e^{-rT}V_T)$, where $e^{-rT}$ is the relevant risk-nuetral discount factor for a constant short rate $r$.

Due to the role of the martingale measure, this approach to valuation is also often called the martingale approach, or - since under the martingale measure all risky asset drift with riskless short rate - the risk-nuetral valuation approach. The second term might, for our purposes, be the better one because in numerical applications, one 'simply' lets the risk factors (stochastic processes) drift by the risk-neutral short rate. One does not habe to deal with the probability measures directly for our applications - they are what theoretically justifies the central theoretical results applied and the technical approach implemented.

Finally, consider market completeness in the general mareket model:

The market model $\mathbf{M}$ is complete if it is arbtrage-free and if every contingent claim is attainable.

Suppose that the market model $\mathbf{M}$ is arbitrage-free. The market model is complete if and only if $\mathbf{M}$ is singleton; i.e., if there is a unique $P$-equivalent martingale measure.

## Risk-Neutral Discounting

This section develops a Python class for risk-neutral discounting. However, it pays to first have a closer look at the modeling and handling of relevant dates for a valuation.

### Modeling and Handling Dates

For valuation purposes, one typically divides the time intervals between today and the final dates of the general market model $T$ into discrete time intervals. These time intervals can be homogeneous (i.e., of equal length), or they can be heterogeneous (i.e., of varying length). A valuation library should be able to handle the more geenral case of hetero geneous time intervals, since the simpler case is then automatically included. The code works with lists of dates, assuming that the smallest relevant time interval is one day. This implies that intraday events are considered irrelevant, for which one would have to model time in addition to dates.

To compile a list of relevant dates, one can take one of two approaches: cosntructing a list of concrete dates (e.r., as `datetime` object in Python) or of year fractions (as decimal numbers).

In [4]:
import numpy as np
import pandas as pd
import datetime as dt

In [5]:
from pylab import mpl, plt
import seaborn as sns

sns.set()
mpl.rcParams['font.family'] = 'serif'
%matplotlib inline

In [6]:
import sys
sys.path.append('./dx')

In [10]:
# the following two definitions of dates and fractions are roughly equivalent.

dates = [dt.datetime(2020, 1, 1), dt.datetime(2020, 7, 1), dt.datetime(2021, 1, 1)]
dates

[datetime.datetime(2020, 1, 1, 0, 0),
 datetime.datetime(2020, 7, 1, 0, 0),
 datetime.datetime(2021, 1, 1, 0, 0)]

In [8]:
(dates[1] - dates[0]).days / 365.

0.4986301369863014

In [9]:
(dates[2] - dates[1]).days / 365.

0.5041095890410959

In [11]:
fractions = [0.0, 0.5, 1.0]

In [12]:
# sometimes it is necessary to get year fractions out of a list of dates.
# the function get_year_deltas() does the job


# get_year_deltas.py

# import numpy as np

# def get_year_deltas(date_list, day_count=365):
#     ''' Return vector of floats with day deltas in year fractions.
#     Initial value normalized to zero.

#     Parameters
#     ==========
#     date_list : list or array
#         collection of datetime objects
    
#     day_count : float
#         number of days for a year
#         (to account for different conventions)

#     Results
#     =======
#     delta_list: array
#         year fractions
#     '''


#     start = date_list[0]
#     delta_list = [(date - start).days / day_count
#                 for date in date_list]

#     return np.array(delta_list)

In [15]:
from dx.get_year_deltas import get_year_deltas

In [16]:
get_year_deltas(dates)

array([0.        , 0.49863014, 1.00273973])

### Constant Short Rate

The exposition focuses on the simplest case for discounting by the short rate; the case where the hort rate is constant through time. Many option pricing models, like the ones of Black-Scholes-Merton (1973), Merton (1976), or Cox-Ross-Rubinstein (1979), make this assumption. The general discount factor as of today, given a future date $t$ and a constant short rate of $r$, is given by $D_0(t)=e^{-rT}$.

The discount factors can also be interpreted as the value of a unit zero-coupon (ZCB) as of today, maturing at $t$. Given two dates $t\ge s \ge 0$, the discount factor relevant for discounting from $t$ to $s$ is then given by the equiation
$$D_s(t)=D_0(t)/D_0(s)=e^{-rt}/e^{-rs}=e^{-rt}\cdot e^{rs}=e^{-r(t-s)}$$

In [17]:
# dx.constant_short_rate is best illustrated by a simple, concrete exmaple.
# the mian result is a two-dimensional ndarray object containing paris of a datetime object and the relevant discount factor.

from dx.constant_short_rate import constant_short_rate

In [30]:
csr = constant_short_rate('csr', 0.05)

In [19]:
csr.get_discount_factors(dates)

array([[datetime.datetime(2020, 1, 1, 0, 0), 0.9510991280247174],
       [datetime.datetime(2020, 7, 1, 0, 0), 0.9753767163648953],
       [datetime.datetime(2021, 1, 1, 0, 0), 1.0]], dtype=object)

In [21]:
deltas = get_year_deltas(dates)
deltas

array([0.        , 0.49863014, 1.00273973])

In [22]:
csr.get_discount_factors(deltas, dtobjects=False)

array([[0.        , 0.95109913],
       [0.49863014, 0.97537672],
       [1.00273973, 1.        ]])

## Market Environments



A market environment mainly consits of three dictionaries to stroe the following types of data and Python objects:

<b>Contants</b>

These can be model parameters or option maturity dates.

<b>Lists</b>

These are collections of objects in general, like a list of objects modeling (risky) securities.

<b>Curves</b>

These are objects for discounting; e.g., an instance of the `dx.constant_short_rate` class.

In [23]:
from dx.market_environment import market_environment

In [24]:
me = market_environment('me_gbm', dt.datetime(2020, 1, 1))

In [26]:
me.add_constant('initial_value', 36.)
me.add_constant('volatility', 0.2)
me.add_constant('final_date', dt.datetime(2020, 12, 31))
me.add_constant('currency', 'EUR')
me.add_constant('frequency', 'M')
me.add_constant('paths', 10000)
me.add_curve('discount_curve', csr)

In [27]:
me.get_constant('volatility')

0.2

In [28]:
me.get_curve('discount_curve').short_rate

0.05

A major advantage of this object-oriented modeing appraoch is that instances of the `dx.constant_short_rate` class can line in multiple environments. Once the instance is updated - for example, when a new constant short rate is set - , all the instances of the `dx.market_environment` class containing that particular instances of the discounting class will be updated automatically.